# Test and results.

In [1]:
from markovify.HMM import HMM
from markovify.ViterbiDecoder import ViterbiDecoder

from nltk.corpus import brown
from math import ceil, floor

def recursive_len(item):
    count = 0
    for subitem in item:
        count += len(subitem)
    return count

In [2]:
corpus = brown.tagged_sents(categories='news')
size = len(corpus)

corpus_train = corpus[:floor(size*9/10)]
corpus_test = corpus[floor(size*9/10):]

print(len(corpus_train))
print(len(corpus_test))

4160
463


In [3]:
model = HMM()
model.train(corpus_train)
#print(model.a)
#print(model.b)

decoder = ViterbiDecoder(model)

In [4]:
# result = decoder.viterbi(corpus_test)
